In [ ]:
import pandas as pd

charge_capture_df = pd.read_excel(r"D:\payroll\Charge Capture - Jul 20th - Aug 2nd.xlsx")
conpany_roaster = pd.read_excel(r"D:\payroll\Company_Roster_1754578475.xlsx", skiprows=2)

CC_used_col = ["Provider", "CPT Codes", "Charge Status"]
CR_used_col = ['Name', "Manager", 'State/Region']

CC_filtered_df = charge_capture_df[CC_used_col]
CR_filtered_df = conpany_roaster[CR_used_col]

# Rename column safely
CR_filtered_df = CR_filtered_df.rename(columns={'Name': 'Provider'})

grouped_CC = CC_filtered_df.groupby('Provider')
grouped_CR = CR_filtered_df.groupby('State/Region')

In [6]:
from rapidfuzz import process, fuzz

def match_providers(company_roster_providers, charge_capture_providers, threshold=75):
    matched = []
    unmatched = []
    already_matched_charge_capture_names = set()

    # Create mappings for normalized to original names
    company_roster_map = {str(provider).strip().lower(): provider for provider in company_roster_providers}
    charge_capture_map = {str(provider).strip().lower(): provider for provider in charge_capture_providers}

    # Normalize the lists
    company_roster_providers = list(company_roster_map.keys())
    charge_capture_providers = list(charge_capture_map.keys())

    for current_threshold in range(90, threshold - 1, -5):  # Iterate thresholds inclusively
        for company_roster in company_roster_providers:
            # Skip if already matched
            if company_roster_map[company_roster] in [item['company_roster'] for item in matched]:
                continue

            # Filter out already matched charge capture names
            available_charge_capture_providers = [
                name for name in charge_capture_providers if name not in already_matched_charge_capture_names
            ]
            match_name, score, _ = process.extractOne(
                company_roster,
                available_charge_capture_providers,
                scorer=fuzz.ratio
            )

            if score >= current_threshold:
                matched.append({
                    'company_roster': company_roster_map[company_roster],  # Use original name
                    'charge_capture_name': charge_capture_map[match_name],  # Use original name
                    'score': score
                })
                already_matched_charge_capture_names.add(match_name)
            else:
                if company_roster_map[company_roster] not in [item['company_roster'] for item in unmatched]:
                    unmatched.append({
                        'company_roster': company_roster_map[company_roster],  # Use original name
                        'charge_capture_name': charge_capture_map[match_name],  # Use original name
                        'score': score
                    })
    filtered_unmatched = [item for item in unmatched if item['company_roster'] not in [m['company_roster'] for m in matched]]
    unmatched = filtered_unmatched
    return {"matched": matched, "unmatched": unmatched}

In [150]:
import re
cpt_pattern = re.compile(r'\b993\d{2}\b')  # Example pattern for 6-digit codes starting with 9930
Region_data = pd.DataFrame(columns=['Clinician', 'Gross Encounters', '99304', '99305', '99306', '99307', '99308', '99309', '99310','Drafts','CCM_counts',"region"])
for name, group in grouped_CC:
  
    group['CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
    group = group.explode('CPT Codes')
    group['CPT Codes'] = group['CPT Codes'].str.strip()

    group['Charge Status'] = group['Charge Status'].str.strip().str.lower()
    # CCM Counts
    CCM_counts =group[group['CPT Codes']=="44444"]
    CCM_counts = CCM_counts['CPT Codes'].count()

    # Filter rows where 'CPT Codes' is in the target list and 'Charge Status' is 'draft'
    draft_counts = group[
        (group['CPT Codes'].str.match(cpt_pattern, na=False)) & 
        (group['Charge Status'] == 'draft')
    ]

    # Count the rows
    draft_counts = draft_counts['Charge Status'].count()
    # Gross Encounters    
    cpt_grouped = group[group['CPT Codes'].str.match(cpt_pattern, na=False)]
    gross_encounters = cpt_grouped['CPT Codes'].count()

    new_row = {'Clinician': name, 'Gross Encounters': gross_encounters,
               '99304': cpt_grouped['CPT Codes'].str.contains('99304').sum(),
               '99305': cpt_grouped['CPT Codes'].str.contains('99305').sum(),
                '99306': cpt_grouped['CPT Codes'].str.contains('99306').sum(),
                '99307': cpt_grouped['CPT Codes'].str.contains('99307').sum(),
                '99308': cpt_grouped['CPT Codes'].str.contains('99308').sum(),
                '99309': cpt_grouped['CPT Codes'].str.contains('99309').sum(),
                '99310': cpt_grouped['CPT Codes'].str.contains('99310').sum(),
                'Drafts': draft_counts,
                'CCM_counts': CCM_counts}
    Region_data.loc[len(Region_data)] = new_row
# 10 1 50 3 2 2

In [151]:
unmatched_providers = []
matched_providers = []
Regional_Dashboard=pd.DataFrame(columns=["Region","RDO","RDS","Gross Encounters","Gross Consents","Gross Drafts"])

for name, group in grouped_CR:
    print(f"State/Region: {name}")
    manager_group=group.groupby('Manager')
    for manager_name, manager_group in manager_group:
        Clinician_list=Region_data['Clinician']
        Clinician_list2 = manager_group['Provider']
        result=match_providers(Clinician_list2, Clinician_list, threshold=74)
        unmatched_providers.extend(result['unmatched'])
        matched_providers.extend(result['matched'])
        Reagion_Gross_Encounters = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['Gross Encounters'].sum()
        Reagion_Gross_Consents = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['CCM_counts'].sum()
        Reagion_Gross_Drafts = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['Drafts'].sum()    
        print(f"Manager: {manager_name}")
        print("Matched Providers:", [(item['charge_capture_name'], item['company_roster'],item['score']) for item in result['matched']])
        print("Unmatched Providers:", result['unmatched'])
        print("Total Clinicians in Region:", len(Clinician_list2))
        print("Total Matched Providers:", len(result['matched']))
        print("Total Unmatched Providers:", len(result['unmatched']))
        print("clincan:", Clinician_list2.to_list())
        print("\n")
        new_row = {
            "Region": name,
            "RDO": manager_name,
            "RDS": Clinician_list2.to_list(),
            "Gross Encounters": Reagion_Gross_Encounters,
            "Gross Consents": Reagion_Gross_Consents,
            "Gross Drafts": Reagion_Gross_Drafts
        }
        Regional_Dashboard.loc[len(Regional_Dashboard)] = new_row

State/Region: Georgia
Manager: Jillian Mills
Matched Providers: [('Shnita Wiley', 'Shnita Wiley', 100.0), ('Chekeli Daniels', 'Chekeli Daniels', 100.0), ('Mya Williams', 'Mya Williams', 100.0), ('Jessica Bonilla', 'Jessica Bonilla', 100.0), ('Summer Williams', 'Summer Williams', 100.0), ('Cassandra Westbrook', 'Cassandra Westbrook', 100.0)]
Unmatched Providers: [{'company_roster': 'Andreen Hylick', 'charge_capture_name': 'Mareah Lucio', 'score': 46.15384615384615}]
Total Clinicians in Region: 7
Total Matched Providers: 6
Total Unmatched Providers: 1
clincan: ['Shnita Wiley', 'Chekeli Daniels', 'Mya Williams', 'Jessica Bonilla', 'Summer Williams', 'Cassandra Westbrook', 'Andreen Hylick']


Manager: Lee Wilford, Waris Hussain
Matched Providers: [('Jillian Mills', 'Jillian Mills', 100.0)]
Unmatched Providers: []
Total Clinicians in Region: 1
Total Matched Providers: 1
Total Unmatched Providers: 0
clincan: ['Jillian Mills']


State/Region: Illinois
Manager: Eli Perez
Matched Providers: [('

In [152]:
Regional_Dashboard.head(10)

,Region,RDO,RDS,Gross Encounters,Gross Consents,Gross Drafts
0,Georgia,Jillian Mills,"[Shnita Wiley, Chekeli Daniels, Mya Williams, ...",988,43,4
1,Georgia,"Lee Wilford, Waris Hussain",[Jillian Mills],186,9,0
2,Illinois,Eli Perez,"[Leah Gravesen, Kiryn Evans, Connie Zurski, Ol...",1579,86,24
3,Illinois,Kevin Widdis,"[Aretha Johnson, Tamara Baird, Jeanea Morris, ...",761,9,5
4,Illinois,"Lee Wilford, Waris Hussain","[Eli S Perez, Tashica Bruce]",96,2,0
5,Illinois,Revana Yousif,"[Hina Patel, Roshana Brown]",0,0,0
6,Illinois,Tashica Bruce,"[Carmen Taylor-Mason, Adejimi Carrington, Vera...",950,17,182
7,Illinois,"Waris Hussain, Lee Wilford",[Kevin Widdis],73,2,2
8,Indiana,Christina Hay,[Bobby Hill],24,2,0
9,Michigan,Adam Geller,"[Shawanda Minor, Shawn Gorkiewicz, Kathryn McA...",682,32,1


In [153]:
Regional_Dashboard.head(30)

,Region,RDO,RDS,Gross Encounters,Gross Consents,Gross Drafts
0,Georgia,Jillian Mills,"[Shnita Wiley, Chekeli Daniels, Mya Williams, ...",988,43,4
1,Georgia,"Lee Wilford, Waris Hussain",[Jillian Mills],186,9,0
2,Illinois,Eli Perez,"[Leah Gravesen, Kiryn Evans, Connie Zurski, Ol...",1579,86,24
3,Illinois,Kevin Widdis,"[Aretha Johnson, Tamara Baird, Jeanea Morris, ...",761,9,5
4,Illinois,"Lee Wilford, Waris Hussain","[Eli S Perez, Tashica Bruce]",96,2,0
5,Illinois,Revana Yousif,"[Hina Patel, Roshana Brown]",0,0,0
6,Illinois,Tashica Bruce,"[Carmen Taylor-Mason, Adejimi Carrington, Vera...",950,17,182
7,Illinois,"Waris Hussain, Lee Wilford",[Kevin Widdis],73,2,2
8,Indiana,Christina Hay,[Bobby Hill],24,2,0
9,Michigan,Adam Geller,"[Shawanda Minor, Shawn Gorkiewicz, Kathryn McA...",682,32,1


In [1]:
def build_metadata(charge_capture_df, company_roster_df):
    CC_used_col = ["Provider", "CPT Codes", "Charge Status"]
    CR_used_col = ['Name', "Manager", 'State/Region']

    CC_filtered_df = charge_capture_df[CC_used_col]
    CR_filtered_df = company_roster_df[CR_used_col]

    # Rename column safely
    CR_filtered_df = CR_filtered_df.rename(columns={'Name': 'Provider'})

    grouped_CC = CC_filtered_df.groupby('Provider')
    grouped_CR = CR_filtered_df.groupby('State/Region')
    cpt_pattern = re.compile(r'\b993\d{2}\b')  # Example pattern for 6-digit codes starting with 9930
    Region_data = pd.DataFrame(columns=['Clinician', 'Gross Encounters', 'C99304', 'C99305', 'C99306', 'C99307', 'C99308', 'C99309', 'C99310','Drafts','CCM_counts',"manager","region"])
    for name, group in grouped_CC:
    
        group['CPT Codes'] = group['CPT Codes'].astype(str).str.split(',')
        group = group.explode('CPT Codes')
        group['CPT Codes'] = group['CPT Codes'].str.strip()

        group['Charge Status'] = group['Charge Status'].str.strip().str.lower()
        # CCM Counts
        CCM_counts =group[group['CPT Codes']=="44444"]
        CCM_counts = CCM_counts['CPT Codes'].count()

        # Filter rows where 'CPT Codes' is in the target list and 'Charge Status' is 'draft'
        draft_counts = group[
            (group['CPT Codes'].str.match(cpt_pattern, na=False)) & 
            (group['Charge Status'] == 'draft')
        ]

        # Count the rows
        draft_counts = draft_counts['Charge Status'].count()
        # Gross Encounters    
        cpt_grouped = group[group['CPT Codes'].str.match(cpt_pattern, na=False)]
        gross_encounters = cpt_grouped['CPT Codes'].count()

        new_row = {'Clinician': name, 'Gross Encounters': gross_encounters,
                   'C99304': cpt_grouped['CPT Codes'].str.contains('99304').sum(),
                   'C99305': cpt_grouped['CPT Codes'].str.contains('99305').sum(),
                    'C99306': cpt_grouped['CPT Codes'].str.contains('99306').sum(),
                    'C99307': cpt_grouped['CPT Codes'].str.contains('99307').sum(),
                    'C99308': cpt_grouped['CPT Codes'].str.contains('99308').sum(),
                    'C99309': cpt_grouped['CPT Codes'].str.contains('99309').sum(),
                    'C99310': cpt_grouped['CPT Codes'].str.contains('99310').sum(),
                    'Drafts': draft_counts,
                    'CCM_counts': CCM_counts}
        Region_data.loc[len(Region_data)] = new_row
    unmatched_providers = []
    matched_providers = []
    Regional_Dashboard=pd.DataFrame(columns=["Region","RDO","RDS","Gross Encounters","Gross Consents","Gross Drafts"])

    for name, group in grouped_CR:
        manager_group=group.groupby('Manager')
        for manager_name, manager_group in manager_group:
            Clinician_list=Region_data['Clinician']
            Clinician_list2 = manager_group['Provider']
            result=match_providers(Clinician_list2, Clinician_list, threshold=74)
            unmatched_providers.extend(result['unmatched'])
            matched_providers.extend(result['matched'])
            Reagion_Gross_Encounters = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['Gross Encounters'].sum()
            Reagion_Gross_Consents = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['CCM_counts'].sum()
            Reagion_Gross_Drafts = Region_data[Region_data['Clinician'].isin([item['charge_capture_name'] for item in result['matched']])]['Drafts'].sum()    
            new_row = {
                "Region": name,
                "RDO": manager_name,
                "RDS": Clinician_list2.to_list(),
                "Gross Encounters": Reagion_Gross_Encounters,
                "Gross Consents": Reagion_Gross_Consents,
                "Gross Drafts": Reagion_Gross_Drafts
            }
            Regional_Dashboard.loc[len(Regional_Dashboard)] = new_row
    Regional_Dashboard.columns = Regional_Dashboard.columns.str.replace(' ', '_')
    Region_data.columns = Region_data.columns.str.replace(' ', '_')
    for row in Regional_Dashboard.itertuples():
        region = row.Region
        Clinician_list = row.RDS
        manager= row.RDO    
        # Update the 'region' column for all clinicians in the list
        Region_data.loc[Region_data['Clinician'].isin(Clinician_list), 'region'] = region
        Region_data.loc[Region_data['Clinician'].isin(Clinician_list), 'manager'] = manager

    
    # Drop rows with NaN values in the 'region' column
    Region_data.dropna(subset=['region'], inplace=True)
    
    gouped_CC_ByRegion = Region_data.groupby(['region', 'manager'])
    return Regional_Dashboard, unmatched_providers, matched_providers, gouped_CC_ByRegion

In [7]:
import pandas as pd
import re
charge_capture_df = pd.read_excel(r"D:\payroll\Charge Capture - Jul 20th - Aug 2nd.xlsx")
conpany_roaster = pd.read_excel(r"D:\payroll\Company_Roster_1754578475.xlsx", skiprows=2)
Regional_Dashboard, unmatched_providers, matched_providers, gouped_CC_ByRegion = build_metadata(charge_capture_df, conpany_roaster)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28152\2474590619.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Georgia' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Region_data.loc[Region_data['Clinician'].isin(Clinician_list), 'region'] = region
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_28152\2474590619.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Jillian Mills' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Region_data.loc[Region_data['Clinician'].isin(Clinician_list), 'manager'] = manager


In [12]:
for name, group in gouped_CC_ByRegion:
    print(f"Region: {name[0]}, Manager: {name[1]}")
    print(group.head(10))
    print("\n")

Region: Georgia, Manager: Jillian Mills
              Clinician  Gross_Encounters  C99304  C99305  C99306  C99307  \
19  Cassandra Westbrook               130       0      11       2       0   
22      Chekeli Daniels               187       0      32       0       0   
40      Jessica Bonilla               259       0      20       0       0   
55         Mya Williams               178       0      16       0       0   
69         Shnita Wiley                77       0       3       0       0   
72      Summer Williams               157       0       5       1       0   

    C99308  C99309  C99310  Drafts  CCM_counts        manager   region  
19      20      97       0       3           0  Jillian Mills  Georgia  
22       0     155       0       0          18  Jillian Mills  Georgia  
40      14     225       0       0          18  Jillian Mills  Georgia  
55     162       0       0       0           0  Jillian Mills  Georgia  
69      74       0       0       1           3  Jillian